In [3]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --timeout 100

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.7 GB 2.2 MB/s eta 0:20:38
     ---------------------------------------- 0.0/2.7 GB 3.6 MB/s eta 0:12:30
     ---------------------------------------- 0.0/2.7 GB 4.8 MB/s eta 0:09:28
     ---------------------------------------- 0.0/2.7 GB 4.8 MB/s eta 0:09:19
     ---------------------------------------- 0.0/2.7 GB 7.2 MB/s eta 0:06:13
     ---------------------------------------- 0.0/2.7 GB 8.1 MB/s eta 0:05:33
     ---------------------------------------- 0.0/2.7 GB 7.8 MB/s eta 0:05:48
     ---------------------------------------- 0.0/2.7 GB 8.6 MB/s eta 0:05:13
     ---------------------------------------- 0.0/2.7 GB 9.1 MB/s eta 0:04:56
     ---------------------------------------- 0.0/2.7 GB 8.8 MB/s eta 0:05:07
     ---------------------------------------- 0.0/2.7 GB 9.6 MB/s eta 0:04:39
     -------

In [4]:
import torch

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.current_device()

0

In [7]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Ti'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Preprocessing like ImageNet
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'C:/Users/Djole/Desktop/ml/projekat_ml/slike'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test']),
}


batch_size = 32
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=False, num_workers=4),
}

model = models.resnet18(pretrained=True)

num_ftrs = model.fc.in_features
num_classes = len(image_datasets['train'].classes) 
model.fc = nn.Linear(num_ftrs, num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
print("Device: ",device)
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    model.train()
    
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(image_datasets['train'])
    epoch_acc = running_corrects.double() / len(image_datasets['train'])

    print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

model.eval()
test_corrects = 0

with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels.data)

test_acc = test_corrects.double() / len(image_datasets['test'])
print(f'Test Accuracy: {test_acc:.4f}')


C:\Users\Djole\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Djole\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Device:  cuda
Epoch 1/10
Train Loss: 1.2722 Acc: 0.5172
Epoch 2/10


In [9]:
from sklearn import metrics

In [ ]:
import torch
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_dataset = datasets.ImageFolder('C:\\Users\\Djole\\Desktop\\ml\\projekat_ml\\slike\\test', transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return all_preds, all_labels

preds, labels = evaluate_model(model, test_loader)

accuracy = accuracy_score(labels, preds)
f1 = f1_score(labels, preds, average='weighted')
recall = recall_score(labels, preds, average='weighted')
precision = precision_score(labels, preds, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

print("\nClassification Report:")
print(classification_report(labels, preds, target_names=test_dataset.classes))


Accuracy: 0.6509
F1 Score: 0.6484
Recall: 0.6509
Precision: 0.6566

Classification Report:
              precision    recall  f1-score   support

       angry       0.66      0.49      0.56       958
     disgust       0.71      0.50      0.59       111
        fear       0.53      0.40      0.46      1024
       happy       0.85      0.85      0.85      1774
     neutral       0.54      0.68      0.60      1233
         sad       0.51      0.61      0.56      1247
    surprise       0.78      0.77      0.77       831

    accuracy                           0.65      7178
   macro avg       0.65      0.61      0.63      7178
weighted avg       0.66      0.65      0.65      7178

